# Bird Observation Data

### Import libraries and configure environment variable

In [68]:
import requests
import pandas as pd
import os

from dotenv import load_dotenv
load_dotenv()
api_key = os.environ.get('EBIRD_API_KEY')

### GET Requests to eBird API

In [69]:
# Fetches all recent recorded observations in PA
recent_observations_url = 'https://api.ebird.org/v2/data/obs/US-PA/recent'
recent_observations_params = {'key': os.environ.get('EBIRD_API_KEY')}
recent_observations = requests.get(recent_observations_url, params=recent_observations_params)

# Handle potential JSONDecodeError
try:
    recent_observations_data = recent_observations.json()
except requests.exceptions.JSONDecodeError:
    print("Error decoding JSON response for recent_observations:")
    print(recent_observations.text)
    exit(1)

### Create DataFrames

In [70]:
# Create a DataFrame to hold API response data
pa_observations_df = pd.DataFrame(recent_observations_data)

### Transform
- Drop unnecessary columns
- Rename columns for clarity
- Remove duplicate data

In [71]:
# Drop unneeded columns from observation data

# Specify the columns to be dropped
columns_to_drop = ['obsValid', 'obsReviewed', 'subId', 'exoticCategory']
# Drop the specified columns from the DataFrame
pa_observations_df = pa_observations_df.drop(columns=columns_to_drop)

In [72]:
# Rename columns for clarity in observation data
pa_observations_df = pa_observations_df.rename(columns={'speciesCode': 'species_code', 'comName': 'common_name', 'sciName': 'scientific_name', 
                                                        'locId': 'location_id', 'locName': 'location_name', 'obsDt' : 'observation_date', 
                                                        'howMany': 'quantity_observed', 'lat': 'latitude', 'lng': 'longitude', 
                                                        'locationPrivate': 'location_private'})

In [73]:
# Remove duplicate entries from DataFrame
pa_observations_deduplicated_df = pa_observations_df.drop_duplicates(keep='first')
pa_observations_deduplicated_df.to_csv('recent_observations.csv', index=False)

In [74]:
# Transform NaN values in the 'Quantity_Observed' column to a default value of 1
pa_observations_deduplicated_df['quantity_observed'].fillna(1, inplace=True)

In [75]:
print(pa_observations_deduplicated_df.head())

  species_code               common_name         scientific_name location_id   
0       grycat              Gray Catbird  Dumetella carolinensis     L663550  \
1       doccor  Double-crested Cormorant     Nannopterum auritum     L663550   
2       houfin               House Finch    Haemorhous mexicanus   L21120016   
3       rebwoo    Red-bellied Woodpecker    Melanerpes carolinus   L21120016   
4       amegfi        American Goldfinch          Spinus tristis     L601577   

                                       location_name  observation_date   
0                                        Keystone SP  2023-08-14 11:50  \
1                                        Keystone SP  2023-08-14 11:50   
2     Hunters Creek--North Marsh (restricted access)  2023-08-14 11:45   
3     Hunters Creek--North Marsh (restricted access)  2023-08-14 11:45   
4  Tioga-Hammond Lakes--Ives Run Rec. Area--West ...  2023-08-14 11:45   

   quantity_observed   latitude  longitude  location_private  
0          

### Write/append data to CSV files

In [76]:
# Append de-duplicated recent observations data to a CSV file
import csv
pa_observations_deduplicated_df.to_csv('recent_observations.csv', mode='a', header=True, index=False)

### Write observation data from Pandas DataFrame to SQLite database

#### Create SQLite Database

In [77]:
import sqlite3

# Specify column names
column_names = ['Species_Code', 'Common_Name', 'Scientific_Name', 'Location_ID', 'Location_Name',
                'Observation_Date', 'Quantity_Observed', 'Latitude', 'Longitude', 'Location_Private']

connection = sqlite3.connect("pa-observation-data.db")
cursor = connection.cursor()

cursor.execute("DROP TABLE IF EXISTS Observations;")

# Create the table with the appropriate columns
create_table_query = "CREATE TABLE Observations ({})".format(", ".join(column_names))
cursor.execute(create_table_query)

#### Write data to database

In [78]:
# Insert values from the DataFrame into the table
for i in range(len(pa_observations_deduplicated_df)):
    cursor.execute("INSERT INTO Observations VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", pa_observations_deduplicated_df.iloc[i])

connection.commit()
connection.close()

In [79]:
# home = os.environ.get('LOCATION')
# pa_observations_df[(pa_observations_df.Location_ID == home)]

pa_observations_deduplicated_df[(pa_observations_deduplicated_df.common_name == 'Pileated Woodpecker')]

,species_code,common_name,scientific_name,location_id,location_name,observation_date,quantity_observed,latitude,longitude,location_private
90,pilwoo,Pileated Woodpecker,Dryocopus pileatus,L4043684,Gifford Pinchot SP--Beaver Creek Area,2023-08-14 09:25,1.0,40.068702,-76.908423,False


In [80]:
pa_observations_deduplicated_df.dtypes

species_code          object
common_name           object
scientific_name       object
location_id           object
location_name         object
observation_date      object
quantity_observed    float64
latitude             float64
longitude            float64
location_private        bool
dtype: object

### Write observation data from CSV file to PostgreSQL database

In [81]:
from sqlalchemy import create_engine

# PostgreSQL database connection details from .env file
host = os.getenv('PG_HOST')
database = os.getenv('PG_DATABASE')
user = os.getenv('PG_USER')
password = os.getenv('PG_PASSWORD')

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}/{database}', pool_pre_ping=True)

# Type conversion of Location_ID column to remove preceding L, if necessary, to store the number as an integer.
pa_observations_deduplicated_df['location_id'] = pa_observations_deduplicated_df['location_id'].astype(str).apply(lambda x: int(x[1:]) if x.startswith('L') else int(x))

table_name = "recent_observations"
try:
    pa_observations_deduplicated_df.to_sql(table_name, engine, if_exists='replace', index=False)
    print("Data insertion successful!")
except Exception as e:
    print("An error occurred during data insertion:", str(e))

Data insertion successful!
